In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [2]:
def get_HTML_from_URL(URL):
    req = requests.get(URL)                            # Get the Page request from URL
    html_content = req.content                         # Extract the HTML content from the URL
    html_content_bs = bs(html_content, "html.parser")  # Beautify HTML Content
    req.close()                                        # Closing the requested page
    return html_content_bs                             # returning the beautified HTML content

In [3]:
# ==============================================================================
webLink = "https://www.flipkart.com"
searchString = "honor"  # "fastrack watch"
searchString = searchString.replace(' ','')

# ==============================================================================
URL = webLink + "/search?q=" + searchString
searchString_HTML = get_HTML_from_URL(URL)

# ==============================================================================
firstProd_link = searchString_HTML.find('div', {'class' : '_13oc-S'}).a['href']
URL = webLink + firstProd_link
firstProd_HTML = get_HTML_from_URL(URL)

# ==============================================================================
reviews_html = firstProd_HTML.findAll('div', {"class": "col JOpGWq"})
reviews_html = reviews_html[0]

# ==============================================================================
all_reviews_link = webLink + list(reviews_html)[5]['href']   # Get all reviews link
all_Reviews_html = get_HTML_from_URL(all_reviews_link)       # Get HTML code of all reviews

# ==============================================================================
# Number of pages of all comments
footer_content = all_Reviews_html.findAll('div', {'class':"_1AtVbE col-12-12"})[-1]
n_pages = int(footer_content.div.div.span.text.split()[-1])

# Page common link===============================================================
pages_common_link = footer_content.div.div.nav.a['href'][:-1]
page_link = webLink + pages_common_link+str(1) # Get Page Link

# ==============================================================================
commentboxes = get_HTML_from_URL(page_link)   # Get HTML code of a page
n_records = commentboxes.findAll('div', {'class': "_1AtVbE col-12-12"})[4:-1] # Get HTML of all comments in each page




reviews = pd.DataFrame(columns=["Customer_Name", "Rating", "Heading", 
                                     "Comment", "Date","Location"])
#reviews = []
def extract_comments(commentbox):
    
    # Customer_Name -----------------------
    try:
        Customer_Name = commentbox.findAll('p', {'class': "_2sc7ZR _2V5EHH"})[0].text
    except:
        Customer_Name = 'None'

    # Customer_Rating ---------------------   
    try:
        Rating = int(commentbox.find('div', {'class':'_3LWZlK _1BLPMq'}).text[0])
    except:
        Rating = 0

    # Comment_Heading ---------------------    
    try:
        Heading = commentbox.find('p', {'class':'_2-N8zT'}).text
    except:
        Heading = "None"

    # Comment_Text --------------------- 
    try:
        Comment = commentbox.find_all('div', {'class': 't-ZTKy'})[0].text
    except:
        Comment = 'None'

    # Comment_Date ---------------------
    try:
        Date = list(commentbox.find('div', {'class':'row _3n8db9'}).div)[-1].text
    except:
        Date = 'None'

    # Comment_from_location ---------------------
    try:
        Location = list(list(commentbox.find('div', {'class':'row _3n8db9'}).div)[2])[-1].text[2:]
    except:
        Location = 'None'
        
    # Creating Dictionary
    comment_record = {'Customer_Name' : Customer_Name,'Rating' : Rating, 
                      'Heading' : Heading, 'Comment' : Comment, 'Date' : Date, 
                      'Location' : Location}
    
    return comment_record

for j in range(len(n_records)):
    record = n_records[j]
    # Calling function to create a comment record
    comment_record = extract_comments(record)  
    
    # Appending into dataframe
    reviews = reviews.append(comment_record, ignore_index=True)

In [4]:
reviews

,Customer_Name,Rating,Heading,Comment,Date,Location
0,Ravindra Sane,4,Good choice,Good Product..in less Price... applications wo...,5 months ago,Thane
1,Yogini Chafle,4,Worth the money,Value for money display is good at this range ...,5 months ago,Kondhali
2,Flipkart Customer,5,Fabulous!,it's good phone value for money .. good displa...,5 months ago,Thane
3,Seema,5,Brilliant,Flipkart HONOR erdly LIED There is No fingerpr...,5 months ago,Nellore
4,Nandlal Ray,0,Hated it!,First of all I would like to inform you all th...,5 months ago,Kolkata
5,BHUBNISH KUMAR,0,Terrible product,"Nice furnishing and design , camera quality is...",5 months ago,Tira Sujanpur
6,Kumaraswamy Mamidi,5,Fabulous!,Price worthy mobile with a good performance an...,5 months ago,Warangal
7,RAJENDRAN D R,0,Don't waste your money,Friends don't by this phonePlay store is not s...,4 months ago,Aroor
8,visham shukla,0,Unsatisfactory,Please don't buy this phone. Price is cheap on...,5 months ago,Sitapur
9,Javed Iqbal,5,Excellent,Nice Smartphone In a Short budget....This mobi...,5 months ago,Mumbai


In [8]:
reviews.loc[0,'Customer_Name']

'Ravindra Sane'

In [11]:
for i in range(len(reviews)):
    print(reviews.loc[i,'Customer_Name'])

Ravindra Sane
Yogini Chafle
Flipkart Customer
Seema
Nandlal Ray
BHUBNISH KUMAR
Kumaraswamy  Mamidi
RAJENDRAN D R
visham shukla
Javed Iqbal
